# Treshold Handling in Extreme multi-label classification (XMLC)

## Vocabulaire :
- extreme multi-label classification (XMLC)
- Probability Label trees (PLT)
- sparse probability estimates (SPEs)
- Sorting-based threshold optimization (STO)
- online F-measure optimization (OFO)

# I - Présentation du sujet

Ce que nous allons étudier dans ce notebook est la multi-classification de données dans un très grand nombre de classes : **Extreme multi-label classification (XMLC)**. Pour chaque instance (ligne) du dataset, l'objectif est d'identifier les différentes catégories/labels de l'instance. Ceci peut être appliqué par exemple à la classification d'images : pour chaque image (instance), l'objectif est d'identifier les différents éléments (labels) présents sur une photo. Y-a-t-il une table sur la photo (Oui ou Non), une chaise (Oui ou Non), un chat (Oui ou Non), un nuages(Oui ou Non) etc... Les différents labels possibles sont parfois très nombreux : des centaines de milliers voire des millions !

L'enjeu est d'arriver à prédire les labels de chacunes de données au mieux, tout en ayant un complexité d'éxécution raisonnable. Ceci n'est pas évident, surtout si les datasets sont très importants. Avec un dataset de $10^4$ instances, et avec $10^6$ labels à prédire, cela fait $10^{10}$ prédictions à réaliser et potentiellement stoquer ! Nous allons essayer de comprendre comment on peut affronter ce problème.


<br>Les idées et algorithmes présentés dans ce notebook sont en grande partie tirées de l'article :

**Extreme F-Measure Maximisation using Sparse Probability Estimates**, <br>Jasinska, Dembcynski, Busa-Fekete, Pfannschmidt, Klerx, Hullermeier - 2016 <br>
http://proceedings.mlr.press/v48/jasinska16.pdf

D'autre articles ont également été utilisés pour la compréhension : 

**Probabilistic Label Trees for Extreme Multi-label Classification**, <br> Kalina Jasinska-Kobus, Marek Wydmuch, Krzysztof Dembczyński, Mikhail Kuznetsov, and Róbert Busa-Fekete <br>
https://arxiv.org/pdf/2009.11218.pdf

# II - Formalisation du problème

## Instances, labels, prédiction : 
Nous nous intéressons à un problème de XMLC avec **m labels**. La particularité de notre étude est que ce nombre m est très grand.<br> Chaque instance se se note sous forme de vecteur : $\mathbf{x_i}$. Notre dataset contient n instances.<br>
L'information d'appartenance de l'instance $\mathbf{x_i}$ aux différentes catégories est contenue dans le vecteur $\mathbf{y_i}  = (y_{i,1}, ... , y_{i,m}) \in \{0,1\}^m$. Si $y_{i,j} = 1$, cela signifie que l'instance $\mathbf{x_i}$ est "labélisé" $j$. <br>
Notre dataset se note donc : $ \mathcal{D} = \{(\mathbf{x_i}, \mathbf{y_i})\}_{i=1}^n$ <br>


On peut également définir la matrice de taille $ n \times m$ contenant toutes les informations de labels du dataset : $\mathbf{Y} = (\mathbf{y_{1}}, ... , \mathbf{y_{n}})$ 

## Classifieurs :

Les classifieurs seront les modèles permettant de prédire les labels des données. Les prédictions de labels d'une instance i de ces classifieurs seront contenues dans le vecteur : $ \mathbf{\hat{y_i}}  = (\hat{y_{i,1}}, ... , \hat{y_{i,m}}) \in \{0,1\}^m $. <br>
De même que qu'au dessus, nous pouvons définir la matrice de prédiction de taille $ n \times m$ du dataset : $\mathbf{\hat{Y}} = (\mathbf{\hat{y}_{1}}, ... , \mathbf{\hat{y}_{n}})$ <br>

Soit $\eta(\mathbf{x},j)$,  la probabilité que le label j de l'instance **x** soit positive : $\eta(\mathbf{x},j) = \mathbf{P}(y_j=1 \vert \mathbf{x})$. <br>

Dans notre étude nous allons nous intéresser à des classifieurs probabilistiques : ils vont estimer ces probabilités  $\eta(\mathbf{x},j)$ <br>

A partir d'une donnée **x**, les classifieurs probabilistiques estiment les probabilités  $\eta(\mathbf{x},j)$ : on note les résultats ainsi obtenus par les classifieurs: $\hat{\eta}(\mathbf{x},j)$ <br>
$$\hat{\eta} : \chi \times [ m ]  \rightarrow [0,1]$$ <br>

Une fois que les classfieurs ont permis d'obtenir ces valeurs $\eta$, il reste à définir les **thresholds** $\mathbf{\tau}$, qui définissent les valeurs minimales que les $\hat{\eta}$ doivent dépasser pour assigner les labels aux instances. <br>
Ainsi, on définit : $\mathbf{\tau} = [\tau_1, ... \tau_m]$ tels que pour tout label j et instance **x** :

$$ \hat{y}_{i,j} =  \left\{\begin{array}{ll} 0 & \textrm{si } \hat{\eta}(\mathbf{x_i},j) > \tau_j  \\ 1 & \textrm{sinon}\end{array}\right.$$

## Métrique à maximiser : la F-measure :

Dans notre étude, nous souhaitons avoir le plus possible de prédictions justes. Pour cela la métrique que nous allons utiliser est la F-measure :

\begin{align*}
F_M (\mathbf{Y}, \mathbf{\hat{Y}}) & = \frac{1}{m} \sum_{j=1}^m F_s(\mathbf{Y}.j, \mathbf{\hat{Y}}.j)\\
& = \frac{1}{m} \sum_{j=1}^m  \frac{2 \sum_{i=1}^n y_{i,j}\hat{y}_{i,j}}{\sum_{i=1}^n y_{i,j} + \sum_{i=1}^n \hat{y}_{i,j}}
\end{align*}

$\mathbf{Y}.j$ et $\mathbf{\hat{Y}}.j$ sont les colonnes j des matrices respectives $\mathbf{Y}$ et $\mathbf{\hat{Y}}$. <br>
$F_M$ est la F-measure Macro (ou F-score) de toutes nos prédictions : elle concerne tous les labels $F_s$. Elle est la moyenne des F-measure standard. La F-measure standard est spécifiques à un label.

Pour revenir sur les trasholds $\tau_j$, l'objectif est est donc de trouver : 
$$ \mathbf{\tau^*} = \underset{\mathbf{\tau} \in [0,1]^m}{\operatorname{argmax}} F(\mathbf{\tau}, \mathbf{\hat{\eta}})$$

Implémentons la F_measure standard sur un exemple simple : 

In [1]:
import math
import numpy as np
import pandas as pd
import time

In [2]:
def F_measure_standard(y,y_hat):
    if (np.sum(y)+np.sum(y_hat)) == 0:
        return False
    else:
        return 2*np.dot(y,y_hat.T)/(np.sum(y)+np.sum(y_hat))
### Complexité : n

y = np.array([1,0,0,1])
y_hat = np.array([1,0,0,1])
print('Standard F-measure pour un prédiction parfaite (4/4) ', F_measure_standard(y, y_hat))
print('\n')
y = np.array([1,0,0,1])
y_hat = np.array([0,1,1,0])
print('Standard F-measure pour un prédiction très mauvaise (0/4) : ', F_measure_standard(y, y_hat))
print('\n')
y = np.array([1,0,0,1])
y_hat = np.array([1,1,0,1])
print('Standard F-measure pour un prédiction intermédiaire (3/4): ', F_measure_standard(y, y_hat))

Standard F-measure pour un prédiction parfaite (4/4)  1.0


Standard F-measure pour un prédiction très mauvaise (0/4) :  0.0


Standard F-measure pour un prédiction intermédiaire (3/4):  0.8


La F-measure standard varie entre 0 et 1.<br>
0 correspond à un prédiction très mauvaise, et 1 correspond à une prédiction parfaite. 

La F-measure Macro étant la moyenne de toutes les F-measures standard pour chaque label, elle possède les même propriétés.

In [3]:
def F_measure_macro(Y, Y_hat):
    nb_labels = len(Y[0])
    result = 0
    nb_no_result = 0
    for j in range(nb_labels):
        y = Y[:,j]
        y_hat = Y_hat[:,j]
        res = F_measure_standard(y,y_hat)
        if res :
            result = result +  res
        else:
            nb_no_result += 1
#         print('Standard F-measure of label ' + str(j) + ' : ', res)
    if nb_labels-nb_no_result >0:
        return (result/(nb_labels-nb_no_result))
    else :
        return 1
### Complexité : n x m

Y = np.array([[1,0],[0,1],[0,0],[1,0]])
print('Y :\n', Y)
Y_hat = np.array([[1,0],[0,1],[0,0],[1,1]])
print('\nY_hat :\n', Y_hat)
print("\nNombre d'instances : ", len(Y))
print("Nombre de labels : ", len(Y[0]), '\n')
print('Macro F-measure :', F_measure_macro(Y, Y_hat))

Y :
 [[1 0]
 [0 1]
 [0 0]
 [1 0]]

Y_hat :
 [[1 0]
 [0 1]
 [0 0]
 [1 1]]

Nombre d'instances :  4
Nombre de labels :  2 

Macro F-measure : 0.8333333333333333


**Notre objectif sera de maximiser la Macro F-measure.** Cela revient à maximiser chaque Standard F-measure de chaque label.

# III - Présentation du dataset utilisé

Nous allons utiliser le dataset **eurlex-4k** qui est issue d'un concours Kaggle (https://www.kaggle.com/c/extreme-classification-eurlex/data?select=EURLex-4K.zip).<br>
Ces données ont été reprises dans ce GitHub (https://github.com/mwydmuch/napkinXC) dans lequel est crée la libraire napkinXC qui crée des models de prédictions pour des datasets avec un nombre de classes très élevé. <br>
Ce dataset (un peu abstrait) contient 4000 labels. <br>
Nous allons donc utiliser la librairie napkinXC pour charger les données.

Ce chargement des données se fait dans le fichier python load_data_napkinxc.py, car quelques pré-traitements inutiles à la compréhension de l'étude y sont fait.

In [4]:
from load_data_napkinxc import X_train_napxinxc, Y_train_napxinxc, X_test_napxinxc, Y_test_napxinxc # Array dans un format particulier spécifique à npakinxc 
from load_data_napkinxc import X_train_arr_tot, X_test_arr_tot, Y_train_arr_tot, Y_test_arr_tot

## Exploration du dataset

### X data

In [5]:
print('Shape X_train : ', (X_train_arr_tot).shape)
print(X_train_arr_tot)

Shape X_train :  (15539, 5000)
[[0.084556 0.138594 0.094304 ... 0.       0.       0.      ]
 [0.050734 0.762265 0.754431 ... 0.       0.       0.      ]
 [0.101468 0.138594 0.377215 ... 0.       0.       0.      ]
 ...
 [0.050734 0.346484 0.188608 ... 0.       0.       0.      ]
 [0.084556 2.39074  0.848735 ... 0.       0.       0.      ]
 [0.084556 0.20789  0.282912 ... 0.       0.       0.      ]]


X_train_arr_tot est donc une matrice contenant les informations de X du dataset. Nous avons donc 15539 instance avec chacunes 5000 features qui sont des floats.

### Y data

In [6]:
print('Shape Y_train : ', Y_train_arr_tot.shape)
print(Y_train_arr_tot)
# print('Y_train item example : ', Y_train[0])

Shape Y_train :  (15539, 3993)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Y_train_arr_tot est également un matrice contenant les informations de label de chaque instance. L'élément i,j de cette matrices sera de 1 si l'instance $\mathbf{x_i}$ est labélisée j, et 0 sinon.

On voit comme l'indique le nom du dataset l'indique (eurlex-4k) qu'il y a environ 4000 labels (3993) dans ce dataset.


Regardons les labels d'une instance : 

In [7]:
def lab_of_instance(y_arr, label):
    return np.where(y_arr[label]==1)
print(lab_of_instance(Y_train_arr_tot, 0))

(array([ 446,  521, 1149, 1249, 1265, 1482]),)


L'instance 0 possède ainsi les labels 446, 521, 149, 1249, 1265 et 1482.

Pour la suite, nous utiliserons parfois seulement une partie (3500 instances) de ce dataset afin d'avoir des temps d'éxécution plus courts. C'est pourquoi nous définissons le variables suivantes.

In [8]:
X_train_arr = X_train_arr_tot[:3500,:]
Y_train_arr = Y_train_arr_tot[:3500,:]

<div class= "alert alert-success", style="font-size:17px;">

Nous allons maintenant nous intéresser à la construction des classifieurs. Dans cette étude elle se déroule en 2 étapes : <br>
<ol>
<li style="font-weight:bold ;"> Etape 1 : Contrsuire le meilleur estimateur des $\eta$.
<li style="font-weight:bold ;"> Etape 2 : Trouver les meilleurs thresholds $\tau$ pour passer de $\hat{\eta}$ à y.
</ol>
    
Dans ce notebook, nous allons parler rapidement de la première partie, puis nous allons nous concentrer sur la deuxième partie
</div>

# IV - Etape 1 : Création des modèles pour obtenir les estimateurs $\hat{\eta}$ :

## Création d'un estimateur binaire spécifique à un label.
Une des solutions possibles pour créer un modèle de prédiction global est de créer un modèle de classification binaire par label.
Par exemple, avec notre dataset, on crérait un modèle par colonne (3993).

Nous allons ici créer un réseau de neuronnes très simple s'entrainant avec une loss binary crossentropy sur un label de notre dataset. Nous noterons par la suite ce label (ou la colonne correspondante) **j**. Ce modèle renvoit pour chaque instance la probabilité que cette instance soit "labélisée" j.

In [9]:
from sklearn.datasets import make_circles
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

# Création du modèle
def train_NN_model_specific_label(label, x_train, y_train):
    t = time.time()
    model = Sequential()
    model.add(Dense(50, input_dim=5000, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    # Entrainement du modèle
    model.fit(x_train, y_train[:,label], epochs=2, verbose=0)
    print("Temps d'entrainement d'un modèle binaire (secondes): ", time.time()-t)
    return model

Using TensorFlow backend.


In [10]:
label = 8 # label pour lequel on crée le modèle
model = train_NN_model_specific_label(label, X_train_arr_tot, Y_train_arr_tot) # Création et entrainement du modèle
pred_probas_train = model.predict(X_train_arr_tot)[:,0] # Prédictions des eta
print(pred_probas_train)

Temps d'entrainement d'un modèle binaire (secondes):  2.64646053314209
[2.2637137e-12 2.5284579e-01 1.6443070e-36 ... 1.9969450e-02 0.0000000e+00
 6.1115049e-15]


Le modèle nous retourne donc ici le vecteur $\hat{\eta}(\mathbf{X},j) = \mathbf{P}(y_j=1 \vert \mathbf{X})$ avex $\mathbf{X}$ qui est X_train_arr_tot et j qui est le label considéré.

### De la prédiction de probabilité à la prédiction :

Pour pouvoir savoir si notre modèle est peformant, il faut savoir à partir de quelle valeur de $\hat{\eta}(x,j)$ on considère que x est labélisé j. Cette valeur correspond à $\tau_j$ défini en partie II. 

On s'attardera la construction lors de l'explication de l'étape 2 en partie V, mais on peut définir pour l'instant un fonction simple qui à partir d'une liste de valeurs de probabilité $\eta$ de taille n crée une liste de prédiction de taille n spécifique à un label. Cette liste donne pour chaque instance 1 si sa probabilité est au dessus d'un certain threshold $\tau_j$ et 0 sinon.

In [11]:
def pred_from_proba_one_label(proba_list, tresh_label): 
    prediction = np.zeros(len(proba_list))
    for j, p in enumerate(proba_list):
        if p>tresh_label:
            prediction[j] = 1
    return prediction
## Complexité  n

La fonction pred_from_proba_one_label est spécifique à un label, on peut aussi créer une fonction qui à partir de la matrice totale de probabilitée, et du vecteur de thresholds $\mathbf{\tau}$ qui contient tous les thresholds $\tau_{j}$ de nos labels, renvoit un grande matrice de prédiction pour notre dataset.

In [12]:
def pred_from_proba_all_labels(proba_list, thresh_list): 
    all_preds = np.zeros(proba_list.shape)
    for i in range(proba_list.shape[1]):
        all_preds[:,i] = pred_from_proba_one_label(proba_list[:,i], thresh_list[i])
    return all_preds

### Résultats pour notre estimateur binaire

On peut donc utiliser la fonction pred_from_proba_one_label pour obtenir les prédictions de labels de notre estimateur binaire en prenant arbitrairement comme treshold 0.5 :

In [13]:
predictions = pred_from_proba_one_label(pred_probas_train, 0.5)
print(predictions)

[0. 0. 0. ... 0. 0. 0.]


Regardons maintenant la F-measure standard de ce label : 

In [14]:
print('F measure standard du label ' + str(label) + ' : ', F_measure_standard(Y_train_arr_tot[:,8], predictions))

F measure standard du label 8 :  0.5380116959064327


On remarque que que le résulat de F measure est moyen. Cela est normal car un modèle type réseau de neuronne n'est pas forcément adapté, et on a pas cherché à l'optimiser / le tuner etc... L'objectif de cet exemple est surtout de mieux comprendre le problème et de mettre en évidence que pour entrainer un modèle prédiction d'un seul label, le temps est déjà conscéquent (1 seconde environ). Donc, si l'on doit créer un modèle par label, le temps d'entrainement sera très long : potentiellement plusieurs heures, voire jours dans le cadre d'un problème avec un m très élevé.

Il est donc important d'utiliser un modèle adapté et optimisé.

## Model PLT : Probabilistic Label Tree :

Un des modèles très souvent utilisé dans le cas d'XMLC est celui d'un PLT. Nous n'allons pas dans ce notebook essayer de créer nous même un PLT mais seulement essayer de comprendre le principe de ce modèle.

Les PLT sont des arbres dont les feuilles correspondent aux labels du problème. La probabilité $\eta(x,j)$ sera déterminée grâce au chemin reliant la racine de l'arbre jusqu'à la feuille correspondant au label j. <br>
Le noeud racine est relié à plusieurs noeuds enfants, divisant ainsi l'espace des labels en des plusieurs sous-ensembles de labels. Puis les noeuds alors crées vont également avoir des noeuds enfants, ce qui va encore créer des sous-ensembles de labels plus petits. Ainsi, on pourra diviser l'ensemble de labels jusqu'à obtenir un seul label à chaque  feuilles.

Chaque noeud est donc un classifieur qui aura pour objectif d'indiquer sur chacunes de ses branches la probabilité $\eta$ que l'instance x qui pourcourt l'arbre ait un label positif au sein des sous-ensembles correspondants à ses enfants respectifs.

On a alors : 
$$ \eta(x,j) =  \prod_{t \in Path(j)}\eta_T(x,t)$$
où $t \in Path(j)$ sont chacunes des branches reliant la racine à la feuille correspondante au label j.

Nous n'allons pas étudier comment les classifieurs des noeuds fonctionnent, seulement essayer de comprendre pourquoi cette structure en arbre peut être intéressante. En effet, à priori, nous avons finalement crée plus que m classifieurs. C'est donc plus de classifieurs que ce que nous aurions fait avec une utilisation de plusieurs estimateurs binaires. Alors pourquoi ces arbres PLT sont intéressants ?

Lorsque l'on étudie les différents labels probables d'une instance x en utilisant un PLT : on parcourt l'arbre, et on réalise la multiplication des probabilités présentes sur les branches. Assez rapidement, on va atteindre des noeuds dont la probabilité est très faible. Cela signifie que la probabilité que x ait un label dans le sous-ensemble de labels correspondant à ce noeud est très faible. Ainsi, l'étude de tous les noeuds issue de ce dernier n'est pas intéressante. Grâce à ce fonctionnement, si on se fixe un treshold de probabilité à partir duquel on considère que les labels à probabilité inférieure à cette valeur sont trop improbables, on peut réduire considérablement le nombre de classifieurs que l'on utilise. En pratique, avec un treshold raisonable on utilise moins de m classifieurs pour prédire les $\hat{ \eta}$ d'une instance. C'est pour cela que les PLT sont intéressants dans les problèmes d'XMLC.

## Chargement du modèle PLT de NapkinXC

La librairie **napkinXC** nous permet de créer des modèles de prédictions PLT pour les datasets avec un nombre de classes très élevé. <br>
Nous allons utiliser ces modèles tels quels.

In [15]:
from napkinxc.models import PLT
from napkinxc.measures import precision_at_k

In [16]:
plt = PLT("eurlex-model") # Création d'un modèle type PLT
plt.fit(X_train_napxinxc, Y_train_napxinxc) # Entrainement du modèle sur tout le train dataset

On peut ensuite utiliser ce modèle pour prédire les $\eta$. <br>
La librairie permet à l'utilisateur de préciser le treshold maximal à partir duquel il n'a plus besoin des prédictions. Par exemple, ci-dessous, si un label est estimé valable à moins de 20%, sa probabilité ne sera pas stoquée. 

In [17]:
t = time.time()
preds_proba = plt.predict_proba(X_train_napxinxc[:3500,:], threshold=0.2)
print("Temps d'éxécution (secondes): ", time.time()-t)
print(preds_proba[0])

Temps d'éxécution (secondes):  0.618349552154541
[(1482, 0.6870196046520064), (1249, 0.6216845654287004), (446, 0.5513888687700165), (521, 0.4663520746994896), (1149, 0.20705874246714603)]


On a donc 4 labels pour lesquel la probabilité que l'instance $X[0]$  soit labélisé est supérieure à 0,2.

Si l'on décide d'accéder à toutes les prédictions pour tous les labels (treshold de 0), les temps d'obtention sont beaucoup plus importants.

In [18]:
t = time.time()
preds_proba = plt.predict_proba(X_train_napxinxc[:3500,:], threshold=0, top_k=0)
print("Temps d'éxécution (secondes): ", time.time()-t)

Temps d'éxécution (secondes):  7.961107492446899


In [19]:
print("Taille de la liste preds_proba : ", len(preds_proba))
print("Taille de chaque item de preds_proba : ", len(preds_proba[0]))
print("Extrait d'un item de preds_proba : ")
print(preds_proba[0][:10])

Taille de la liste preds_proba :  3500
Taille de chaque item de preds_proba :  3993
Extrait d'un item de preds_proba : 
[(1482, 0.6870196046520064), (1249, 0.6216845654287004), (446, 0.5513888687700165), (521, 0.4663520746994896), (1149, 0.20705874246714603), (1265, 0.16456291098252113), (591, 0.06715901876819737), (43, 0.030617532537065877), (201, 0.025018042184171134), (2223, 0.022965823915800654)]


Les résultats sont renvoyés comme un tableau où, chaque ligne correspond à une instance, et chaque colonne contient un doublet (label, probabilité). Les colonnes sont ordonnées par ordre de probabilité. Par exemple, en première colonne et première ligne se trouvent les informations du label le plus probable pour la première instance ainsi que sa probabiilité.

Ce pré-tri est une spécificité de la librairie napkinXC. Pour la suite de notre étude nous allons utiliser les prédictions de probabilité pour tous les labels et nous allons considérer que les prédictions de base, ne sont pas classés. Nous allons donc re-trier ces prédictions dans un ordre plus intuitif, où la $j^{ème}$ colonne continent les valeurs de probabilités du label $j$.<br>
Nous allons donc créer le tableau Y_proba qui contient les valeurs $\eta$ du dataset obtenues par le classifieur. Sur la ligne i et la colonne j de Y_proba se trouve $\eta_{i,j}$

In [20]:
Y_proba_sort_index = [ sorted(preds_proba[i], key=lambda r: r[0]) for i in range(len(preds_proba))]
Y_proba = np.array([[prob for (ide,prob) in Y_proba_sort_index[i]]for i in range(len(Y_proba_sort_index))])
print('Shape Y_proba : ', Y_proba.shape)
print(Y_proba)

Shape Y_proba :  (3500, 3993)
[[1.85232331e-05 4.28990773e-04 2.40944964e-05 ... 8.48217732e-06
  7.28495341e-06 1.05442195e-05]
 [1.78664761e-05 1.14571933e-02 4.47722711e-04 ... 1.08242077e-05
  5.82864890e-06 8.76953273e-06]
 [2.57057006e-05 1.59987784e-03 4.26426938e-05 ... 5.47150008e-05
  2.71647065e-05 1.40854832e-05]
 ...
 [2.82723547e-06 7.07738799e-04 1.97562389e-06 ... 4.62966707e-07
  2.99457979e-07 1.92104717e-06]
 [1.35577909e-05 4.13071840e-01 6.14580439e-04 ... 1.62272295e-05
  1.23989849e-05 7.11139961e-06]
 [3.06055951e-06 1.90773925e-04 2.89767660e-06 ... 7.69885482e-06
  9.90820122e-07 1.76674407e-06]]


### Evaluation des résultats obtenus :

Nous pouvons maintenant tester les prédictions de ce modèle NapkinXC en utilisant la fonction F_measure_macro définie plus haut.<br>
Nous considèreront arbitrairement des tresholds à 0,3 pour chaque label.

In [21]:
thresh_list = np.ones(Y_proba.shape[1])*0.3
t = time.time()
preds = pred_from_proba_all_labels(Y_proba, thresh_list)
print("Temps d'éxécution (secondes): ", time.time()-t)
print("F_measure macro avec un threshold de 0.3 pour chaque label :", F_measure_macro(Y_train_arr, preds))

Temps d'éxécution (secondes):  2.1136634349823
F_measure macro avec un threshold de 0.3 pour chaque label : 0.8123877095745675


On se rend compte que attribuer à chaque instance ses prédictions prend déjà quelques secondes. On se rend donc compte que lorsque que l'on cherchera à trouver le meilleur vecteur $\mathbf{\tau}$, il sera important d'essayer le moins possible.

# V - Etape 2 : Trouver les thresholds $\tau$ optimaux pour passer de $\hat{\eta}$ à y efficacement

## Impact de $\tau$ sur la F-measure

La F-measure est la métrique que l'on cherche à maximiser. La maximiser revient à maximiser les Standard F-measure de chaque label. Notre objectif est donc de trouver pour chaque label **j**, le bon threshold $\mathbf{\tau_j}$ qui permette de maximiser la F-measure standard de ce label.

Regardons ensemble l'impact de ce treshold sur notre F-measure Macro : 

In [22]:
preds = pred_from_proba_all_labels(Y_proba, np.ones(Y_proba.shape[1])*0.05)
print('F measure avec threshold de 0.1 : ', F_measure_macro(Y_train_arr, preds))
preds = pred_from_proba_all_labels(Y_proba, np.ones(Y_proba.shape[1])*0.3)
print('F measure avec threshold de 0.3 : ', F_measure_macro(Y_train_arr, preds))
preds = pred_from_proba_all_labels(Y_proba, np.ones(Y_proba.shape[1])*0.5)
print('F measure avec threshold de 0.5 : ', F_measure_macro(Y_train_arr, preds))

F measure avec threshold de 0.1 :  0.7468708710520515
F measure avec threshold de 0.3 :  0.8123877095745675
F measure avec threshold de 0.5 :  0.6771001773087131


On remaque que en fonction du threshold que l'on applique la F-measure varie. Nous allons donc nous intéresser au différents moyen de trouver le threshold optimal.

# V-1 Méthodes "de base" pour optenir les $\mathbf{\tau}$ optimaux :

## Méthode 1, Méthode naïve : Essaie de plusieurs thresholds avec un pas :
On peut naïvement chercher le threshold donnant la meilleure F-Measure entre 0 et 1 avec un pas donné. Créons une fonction réalisant cela pour un label :

In [23]:
def naive_threshold_optimisation_one_label(y_true, proba_pred, pas):
    best_f_meas = 0
    best_thresh = 1
    for i in np.arange(0,1,pas):
        f_meas_tmp = F_measure_standard(y_true, pred_from_proba_one_label(proba_pred, i))
        if f_meas_tmp > best_f_meas:
            best_thresh = i
            best_f_meas = f_meas_tmp
    return (best_f_meas, best_thresh)

La complexité de cette fonction est : $\frac{1}{pas}\times n$

In [24]:
label = 4
naive_threshold_optimisation_one_label(Y_train_arr[:,label], Y_proba[:,label], 0.1)

(0.9534883720930233, 0.2)

Nous pouvons alors réaliser cela pour tous les labels :

In [25]:
def give_all_thresholds_naive_solution(y_prob, y_true):
    t = time.time()
    threshs = np.zeros(Y_proba.shape[1])
    for i in range(len(threshs)):
        if i%1000==0:
            print(i, ' / ', len(threshs))
        threshs[i] = naive_threshold_optimisation_one_label(y_true[:,i], y_prob[:,i], 0.1)[1]
    print("Temps d'éxécution (secondes): ", time.time()-t)
    return threshs
thresholds = give_all_thresholds_naive_solution(Y_proba, Y_train_arr)
print("Liste de threshold", thresholds)

0  /  3993
1000  /  3993
2000  /  3993
3000  /  3993
Temps d'éxécution (secondes):  29.318833827972412
Liste de threshold [0.1 0.3 1.  ... 1.  1.  1. ]


La complexité de cette fonction est de $\frac{1}{pas} \times n \times m$ et le temps d'éxécution est long (quelques dizaines de secondes)

In [26]:
preds = pred_from_proba_all_labels(Y_proba, thresholds)
print('F measure macro : ', F_measure_macro(Y_train_arr, preds))

F measure macro :  0.861175276424519


## Méthode 2, Méthode "solution exacte" (mais de grande complexité) :

Pour être certain d'avoir les $\tau_j$ optimaux, on peut aussi donner à $\tau_j$ itérativemement toutes les valeurs différentes prises par les $\eta_j$, et voir la F-measure la plus élevée.

Si l'on réalise cela pour un label : 

In [27]:
def exact_threshold_solution(y_true, proba_pred):
    best_f_meas = 0
    best_thresh = 1
    for p in proba_pred:
        f_meas_tmp = F_measure_standard(y_true, pred_from_proba_one_label(proba_pred, p))
        if f_meas_tmp > best_f_meas:
            best_thresh = p
            best_f_meas = f_meas_tmp
    return (best_f_meas, best_thresh)

La complexité de cette fonction est $n^2$ 

In [28]:
label = 4
t = time.time()
print(exact_threshold_solution(Y_train_arr[:,label], Y_proba[:,label]))
print("Temps d'éxécution (secondes): ", time.time()-t)

(0.9534883720930233, 0.189021635672496)
Temps d'éxécution (secondes):  2.5765597820281982


Pour un seul label, le temps d'éxécution est déjà de l'ordre de la seconde avec un n relativement petit (3500). Cette solution semble inenvisageable pour trouver les thresholds de tous les labels (4000 labels).

## Méthode 3, Test sur les valeurs de $\eta$ au dessus d'un certain threshold :

A cause du temps d'éxécution de la solution exacte, on ne va plus tester toutes les valeurs de $\eta$, mais seulement les valeurs de $\eta$ qui sont les plus élevées. Pour cela nous allons donc auparavant trier la liste les prédiction $\eta$, puis tester toutes ces valeurs pour $\tau_j$ juqu'à une certaine valeur minimale $\kappa_j$.

La fonction de tri initiale est (pour chaque label) de complexité $n \log{n}$

In [29]:
def sort_threshold_solution(y_true, proba_pred, k):
    best_f_meas = 0
    best_thresh = 1
    sorted_pred = proba_pred.copy()
    sorted_pred = np.sort(sorted_pred)
    if len(proba_pred)>0:  
        i= 1
        while (i< len(sorted_pred)) & (sorted_pred[-i]>k):
            p = sorted_pred[-i]
            f_meas_tmp = F_measure_standard(y_true, pred_from_proba_one_label(proba_pred, p))
            if f_meas_tmp > best_f_meas:
                best_thresh = p
                best_f_meas = f_meas_tmp
            i += 1
        return (best_f_meas, best_thresh)
    else:
        if len(y_true)>0:
            return (0,1)
        else:
            return (1,1)

In [30]:
label=4
sort_threshold_solution(Y_train_arr[:,label], Y_proba[:,label], 0.05)

(0.9534883720930233, 0.189021635672496)

Si l'on prend un threshold initial $\kappa_j$ assez petit, cette méthode à plus de chances de donnef la solution exacte

In [31]:
def give_all_thresholds_sort_solution(y_proba, y_true, k):
    t = time.time()
    treshs = np.zeros(y_proba.shape[1])
    for i in range(len(thresholds)):
        if i%1000==0:
            print(i, ' / ', len(treshs))
        treshs[i] = sort_threshold_solution(y_true[:,i], y_proba[:,i], k)[1]
    print("Temps d'éxécution (secondes): ", time.time()-t)
    return treshs
thresholds = give_all_thresholds_sort_solution(Y_proba, Y_train_arr, 0.05)
print("Liste de threshold", thresholds)

0  /  3993
1000  /  3993
2000  /  3993
3000  /  3993
Temps d'éxécution (secondes):  26.246495723724365
Liste de threshold [0.44939146 0.30559412 1.         ... 1.         1.         1.        ]


In [32]:
preds = pred_from_proba_all_labels(Y_proba, thresholds)
print('F measure macro : ', F_measure_macro(Y_train_arr, preds))

F measure macro :  0.8887356492222076


On remarque que l'on obtient des meilleurs résultats qu'avec la méthode naïve ou l'on teste les thresholds avec un pas donné. D'un point de vue compléxité, cela dépend la répartition des probabilités du classifieur. En pratique sur cet exemple, le temps d'éxécution est du même ordre de grandeur que pour la méthode naïve. <br>
En revanche le threshold initial de tri a beaucoup d'importance. On peut remarquer que si on prend un threshold de 0.1 au lieu de 0.05 comme au dessus, les résultats sont moins bons mais le temps d'éxécution est plus court.

In [33]:
thresholds = give_all_thresholds_sort_solution(Y_proba, Y_train_arr, 0.1)
preds = pred_from_proba_all_labels(Y_proba, thresholds)
print('F measure macro : ', F_measure_macro(Y_train_arr, preds))

0  /  3993
1000  /  3993
2000  /  3993
3000  /  3993
Temps d'éxécution (secondes):  17.701774835586548
F measure macro :  0.8449454911971181


# V-2 Méthodes optimisées pour optenir les $\mathbf{\tau}$ optimaux :

## Méthode STO : Search-based trehsold optimization :

L'objectif de la méthode STO est encore une fois de déterminer les meilleurs thresholds $\tau$, mais cette fois avec un complexité moindre par rapport aux méthodes basique que l'on vient de voir. La méthode STO est basée sur le concepte de SPE (sparse probability estimates). Ces SPE sont en fait un sous ensemble des $\eta$ dans lequel on a gardé seulement les probabilités $\eta$ suffisamment grande. On considère donc que ce sous ensemble suffira à déterminer les thresholds $\tau$ optimaux. Travailler seulement sur les SPE réduira grandement le temps d'éxécution de nos algorithmes.

Les SPE d'une instance **x** que l'on va utiliser sont : 

$$ \hat{s}_{\mathbf{\kappa}}(\mathbf{x}) = \{ \hat{\eta}(\mathbf{x},j) : j \in \hat{\ell_{\kappa}}(\mathbf{x}) \}$$

$$\hat{\ell_{\kappa}}(\mathbf{x}) = \{ j \in [m] : \hat{\eta}(\mathbf{x},j) > \kappa_{j}    \}  $$

où $\mathbf{\kappa} = (\kappa_1, ... , \kappa_m) \in  [0, 1]^m$ et $\kappa_j$ est le threshold pour le label j à partir duquel on utilisera la donnée $\hat{\eta}_(x,j)$ pour trouver le thresholds $\tau_j$ optimal. Bien sûr, les thresholds $\kappa_j$ doivent être inférieurs aux thresholds $\tau_j$ à déterminer. Pour simplifier l'algorithmes on prendra le même thresholds $\kappa$ pour tout le monde : $\kappa_1 = ... = \kappa_m = \kappa$

L'idée de l'agorithme de STO est dans un premier temps de trouver les différents $\hat{s}_{\mathbf{\kappa}}(\mathbf{x})$ et $\hat{\ell_{\kappa}}(\mathbf{x})$ pour chaque instance, puis, comme on a déjà fait, tester ces chacuns des $\eta(x,j)$ retenus comme threshold $\tau_j$ et calculer la F_measure correspondante mais seulement sur les données que l'on aura selectionné.

In [34]:
def give_lkx_skx(proba_x, list_k):
    lkx = []
    skx = []
    for j in range(len(proba_x)):
        proba_tmp = proba_x[j]
        if proba_tmp > list_k[j] :
            lkx.append(j)
            skx.append(proba_tmp)
    return lkx, skx

Etape 1 : Lister les instances à utiliser pour chaque label :

In [35]:
# Chacun de ces éléments est une ensemble d'instances qui répertorient les instances qui seront utilisés pour le calcul de threshold.
def sto_give_sets(y_prob, list_k):
    t = time.time()
    sets_j = []
    for j in range(y_prob.shape[1]):
        sets_j.append([])
    for i in range(y_prob.shape[0]):
        lkx_tmp, skx_tmp = give_lkx_skx(y_prob[i], list_k)
        for lab in lkx_tmp:
            sets_j[lab].append(i)
    print("Temps d'éxécution étape 1 (secondes): ", time.time()-t)
    return sets_j

Etape 2 : Déterminer les thresholds optimaux à partir de ces instances :

In [36]:
def sto_thresholds_from_sets(sets, y_prob, y_true):
    t = time.time()
    treshs = np.zeros(len(sets))
    for j in range(len(sets)):
        probas_j = np.take(y_prob[:,j], sets[j])
        y_j = np.take(y_true[:,j], sets[j])
        _, treshs[j] = exact_threshold_solution(y_j,probas_j)
    print("Temps d'éxécution étape 2 (secondes): ", time.time()-t)
    return treshs

Fonction avec les deux étapes combinées

In [37]:
def sto_both_steps(y_prob, y_true, list_k):
    sets_j = sto_give_sets(y_prob, 0.05*np.ones(Y_proba.shape[1]))
    return(sto_thresholds_from_sets(sets_j, y_prob, y_true))

In [38]:
thresholds = sto_both_steps(Y_proba, Y_train_arr, 0.05*np.ones(Y_proba.shape[1]))
preds = pred_from_proba_all_labels(Y_proba, thresholds)
print('F measure macro : ', F_measure_macro(Y_train_arr, preds))

Temps d'éxécution étape 1 (secondes):  4.641836643218994
Temps d'éxécution étape 2 (secondes):  2.0305562019348145
F measure macro :  0.8886988682894431


Le réultat est similaire à ce que l'on a pu obtenir pour la méthode 3 ce qui est normal puisque les calculs réalisés ont la même logique mathématique. En revance, on a un temps global d'éxécution d'environ 6 secondes. Cela est 4 fois inférieur au temps d'éxécution que l'on avait avec la méthode 3.

## Méthode OFO (Online F-measure Optimization) :

Nous allons étudier une dernière méthode qui permet de trouver les threshold optimaux avec un "faible" temps déxécution . Nous allons pour cela créer un algorithme où le threshold $\tau$ va évoluer au fur et à mesure que l'on explore le dataset.

On a pour le label j :
$$ F_{i,j} =  \frac{2 \sum_{p=1}^i y_{p,j}\hat{y}_{p,j}}{\sum_{p=1}^i y_{p,j} + \sum_{p=1}^j \hat{y}_{p,j}} =  \frac{2 a_{i,j}}{b_{i,j}}$$
avec : 
$$ a_{i,j} = \sum_{p=1}^i y_{p,j}\hat{y}_{p,j}$$,
    print(sets_j) $$b_{i,j} = \sum_{p=1}^i y_{p,j} + \sum_{p=1}^j \hat{y}_{p,j}$$

L'algorithme OFO fixe la valeur du threshold à : $$\tau_{i,j} = \frac{a_{i-1,j}}{b_{i-1,j}}$$

L'idée de l'agorithme est de parcourir le dataset et de faire évoluer la valeur de $\tau_{i,j}$ graĉe aux relations suivantes : 

$$ a_{i,j} = a_{i-1,j} + y_{i,j} \times \hat{y}_{i,j}$$
$$ b_{i,j} = b_{i-1,j} + y_{i,j} + \hat{y}_{i,j}$$

où $\hat{y}_{i,j}$ est calculé grace à $\tau_{i-1,j}$

On peut se demander légitimement pourquoi $\tau_{i,j}$ est fixé à $\frac{a_{i-1,j}}{b_{i-1,j}}$. Cette condition vient de l'étude [Zhao et al, 2013]. Il y est expliqué que lorsque l'on connait les probabilités réelles $\eta$ de tout le dataset, nous avons la relation suivante entre le $\tau$ optimal $\tau^*$ et la F-measure correspondante : 
$$ F^* = F(\tau^*) = 2\tau^*$$

Essayons maintenant de coder cette méthode. On initialisera les premier $\tau_j$ à 0,4. Pour cela les $a_{0,j}$ et $ b_{0,j}$ seront initialisés à 10 et 4 respectivement

In [39]:
def OFO(y_prob, y_true, a_list, b_list):
    t = time.time()
    n, m = y_prob.shape
    list_thresh = [a/b for (a,b) in zip(a_list, b_list)] #Initialisation des thresholds avec les valeurs initiales de a et b
    for i in range(n):
        lkx, _ = give_lkx_skx(y_prob[i], list_thresh) # Les labels de x selon les predictions de probabilités et threshold_i
        true_label_of_x = np.where(y_true[i] == 1) # Les vrais labels de x
        lkx = set(lkx)
        true_label_of_x = set(true_label_of_x[0])
        for j in lkx.union(true_label_of_x):
            a_list[j] += len(lkx.intersection(true_label_of_x))
            b_list[j] += len(lkx) + len(true_label_of_x)
            list_thresh[j] = a_list[j]/b_list[j]
    print("Temps d'éxécution étape 2 (secondes): ", time.time()-t)
    return list_thresh

In [40]:
a_l = 4*np.ones(Y_proba.shape[1])
b_l = 10*np.ones(Y_proba.shape[1])
thresholds = OFO(Y_proba, Y_train_arr, a_l, b_l)

Temps d'éxécution étape 2 (secondes):  2.7430944442749023


In [41]:
preds = pred_from_proba_all_labels(Y_proba, thresholds)
print('F measure macro : ', F_measure_macro(Y_train_arr, preds))

F measure macro :  0.7687405567711114


On remarque que la F_measure est moins bonne qu'avec les autres méthodes, ce qui peut normal puisque ces dernière sont des dolutions plus "exactes". En revanche le temps d'éxécution est plus faible (divisé par 2 environ).